Importing the dependencies

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import zscore

loading the data

In [4]:
# Load the benin data
df = pd.read_csv(r"C:\Users\gasha\Desktop\solar-challenge-week0\data\benin-malanville.csv")
print("Dataset shape:", df.shape)

Dataset shape: (525600, 19)


# 1 Summary Statistics & Missing-Value Report

In [5]:
print("An overview of the dataset: the shape of the dataset")
df.info()

An overview of the dataset: the shape of the dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Timestamp      525600 non-null  object 
 1   GHI            525600 non-null  float64
 2   DNI            525600 non-null  float64
 3   DHI            525600 non-null  float64
 4   ModA           525600 non-null  float64
 5   ModB           525600 non-null  float64
 6   Tamb           525600 non-null  float64
 7   RH             525600 non-null  float64
 8   WS             525600 non-null  float64
 9   WSgust         525600 non-null  float64
 10  WSstdev        525600 non-null  float64
 11  WD             525600 non-null  float64
 12  WDstdev        525600 non-null  float64
 13  BP             525600 non-null  int64  
 14  Cleaning       525600 non-null  int64  
 15  Precipitation  525600 non-null  float64
 16  TModA          525600

In [6]:
print("Missing Value Report:")
print(df.isnull().sum())

Missing Value Report:
Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64


In [7]:
#list any column with >5% nulls
print(f"Columns with > 5% nulls: {[col for col in df.columns if df[col].isnull().sum()/len(df) > 0.05]}")

Columns with > 5% nulls: ['Comments']


In [8]:
print("Description of the numerical columns summary statistics:")
print(df.describe())

Description of the numerical columns summary statistics:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.1

In [9]:
#Exact duplicate rows
print("Duplicated rows:", df.duplicated().sum())


Duplicated rows: 0


# 2 Outlier Detection & Basic Cleaning

In [10]:
# Check for missing values
print("Look for missing values",df.isnull().sum())

Look for missing values Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64


In [ ]:
# list all columns
print("List all columns", df.columns)

List all columns Index(['Timestamp', 'GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS',
       'WSgust', 'WSstdev', 'WD', 'WDstdev', 'BP', 'Cleaning', 'Precipitation',
       'TModA', 'TModB', 'Comments'],
      dtype='object')


In [27]:
# Compute Z-scores for GHI, DNI, DHI, ModA, ModB, WS, WSgust; flag rows with |Z|>3.
z_scores = df[['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']].apply(zscore)
print("z_scores :\n", z_scores )

z_scores :
              GHI       DNI       DHI      ModA      ModB        WS    WSgust
0      -0.730102 -0.639591 -0.733873 -0.723749 -0.723088 -1.322831 -1.187312
1      -0.729800 -0.639591 -0.733873 -0.723749 -0.723088 -1.322831 -1.384442
2      -0.729800 -0.639591 -0.733873 -0.723749 -0.723088 -1.135736 -0.842334
3      -0.729800 -0.639209 -0.733243 -0.723749 -0.723088 -1.198101 -1.039464
4      -0.729498 -0.639209 -0.733243 -0.723749 -0.723088 -1.260466 -1.039464
...          ...       ...       ...       ...       ...       ...       ...
525595 -0.743088 -0.639209 -0.764120 -0.723749 -0.723088 -1.135736 -0.842334
525596 -0.743088 -0.639209 -0.763490 -0.723749 -0.723088 -1.198101 -1.039464
525597 -0.743088 -0.639209 -0.763490 -0.723749 -0.723088 -0.948641 -0.842334
525598 -0.743088 -0.639209 -0.763490 -0.723749 -0.723088 -0.761547 -0.743769
525599 -0.743088 -0.639209 -0.762860 -0.723749 -0.723088 -0.574452 -0.595921

[525600 rows x 7 columns]


In [29]:
# Check outliers for all numeric columns for Z-score analysis
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
z_scores = df[numeric_columns].apply(zscore)
outliers = (z_scores.abs() > 3).sum()  # Count outliers
print("Outliers:\n", outliers)


Outliers:
 GHI                89
DNI                 0
DHI              3738
ModA               27
ModB               63
Tamb                0
RH                  0
WS               3109
WSgust           3500
WSstdev          5653
WD                  0
WDstdev          4765
BP                865
Cleaning          485
Precipitation    1402
TModA              10
TModB              98
Comments            0
dtype: int64


In [30]:
# Check for outliers, especially in sensor readings (ModA, ModB) and wind speed data (WS, WSgust) with |Z|>3.
z_scores = df[['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']].apply(zscore)
outliers = (z_scores.abs() > 3).sum()  # Count outliers
print("Outliers for selected columns:\n", outliers)

Outliers for selected columns:
 GHI         89
DNI          0
DHI       3738
ModA        27
ModB        63
WS        3109
WSgust    3500
dtype: int64
